<a href="https://colab.research.google.com/github/ArtHommage/HommageTools/blob/main/notebooks/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools

In [1]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI from main branch =-
  !git fetch origin
  !git checkout master
  !git reset --hard origin/master
  !git pull origin master

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI from main branch =-
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 60 (delta 33), reused 30 (delta 30), pack-reused 18 (from 1)
Unpacking objects: 100% (60/60), 42.91 KiB | 4.00 KiB/s, done.
From https://github.com/comfyanonymous/ComfyUI
   eade1551..84fdaf7b  master                 -> origin/master
 * [new branch]        annoate_get_input_info -> origin/annoate_get_input_info
 * [new branch]        move_to_trash          -> origin/move_to_trash
 + bef5a067...d58ad2dd yo-lora-trainer        -> origin/yo-lora-trainer  (forced update)
M	.ci/update_windows/update.py
M	.ci/update_windows/update_comfyui.bat
M	.ci/update_windows/update_comfyui_stable.bat
M	.ci/windows_base_files/README_VERY_IMPORTANT.txt
M	.ci/windows_base_files/run_cpu.bat
M	.ci/windows_base_files/ru

Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!pip install transparent_background piexif ultralytics surrealist blend_modes gguf dynamicprompts pygtrans segment_anything dill boto3 fake_useragent redis fal_client replicate onnxruntime albumentations --upgrade
!pip install --upgrade transformers==4.49.0

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-03-26 23:43:56--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb [following]
--2025-03-26 23:43:56--  https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/4f284b8f-4452-49e8-a0c0-0e45c3b6e65e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250326%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250326T234356Z&X-Amz-Expires=300&X-Amz-Signature=1b452ba029a766698d27dd71d0f34c81faf69fea079648d01c5baf00